# JavaScript Event Loop 

## Introduction

- Topic: Understanding the **JavaScript Event Loop**, its need, and how it handles **synchronous** and **asynchronous** code using the **call stack**, **browser APIs**, and **callback queue**.[1]
- Goal: Explain how JavaScript, despite being **single-threaded**, can handle asynchronous operations (timers, events, network calls) without blocking the main thread.[1]

***

## Synchronous vs Asynchronous Code

### Synchronous Code

**Definition:**  
Synchronous code is executed **immediately and in order** as soon as the execution flow reaches a line; it does not make you wait. Each line completes before the next one starts.[1]

**Example idea:**  
```js
console.log("Hi");
console.log("Love");
console.log("Babbar");
```
- When execution reaches each line, the message is printed **on the spot**.
- There is no delay; the behavior is guaranteed as soon as execution hits that line.[1]

**Key Characteristics:**

- Executes **line by line** in sequence.
- No waiting for external operations.
- The result appears immediately once the line is reached.[1]

***

### Asynchronous Code

**Definition:**  
Asynchronous code is code whose behavior does **not** necessarily happen at the exact moment the execution flow reaches that line. Execution proceeds ahead, and the result may appear **later** (not guaranteed at a fixed time).[1]

**Typical examples:**

- `setTimeout`  
- Event listeners (e.g., `onclick`)  
- Network requests (AJAX/fetch)  
- Database connection requests[1]

**Example idea (setTimeout):**
```js
function sayMyName() {
  console.log("Love Babbar");
}

setTimeout(sayMyName, 5000);
```
- Execution reaches `setTimeout`, but `"Love Babbar"` is **not** printed immediately.[1]
- Even with `5000` ms delay, there is **no strict guarantee** that it will run *exactly* after 5 seconds; it may be later (e.g., 10s, 25s), depending on whether the main thread is free.[1]

**Key Characteristics:**

- The execution reaches the asynchronous line, but the effect (callback execution) happens **later**.
- The time delay given (like 5000 ms) is the **earliest** time after which the callback becomes **eligible** to run, not a guarantee it will run exactly then.[1]
- Used heavily for:
  - Timers  
  - UI events  
  - Network I/O  
  - DB operations[1]

***

## Why Asynchronous Code Is Needed (Blocking Problem)

### Blocking

**Blocking** happens when a piece of code prevents the rest of the program from executing until it finishes, causing everything below it to wait.[1]

**Thought experiment (no async, everything synchronous):**
```js
console.log("Start");
// Some long network request
console.log("End");
```
- If the network call is synchronous:
  - `"Start"` prints.[1]
  - Execution reaches the network call and **waits** until response comes.
  - `"End"` is blocked until that response arrives.[1]
- All code below is stuck; the main thread is **blocked**.

In real applications (e.g., front-end web apps):

- Blocking the main thread means the page becomes **unresponsive**.
- UI cannot update, user cannot interact smoothly.[1]

### Need for Asynchronous Handling

To avoid blocking:

- Long or uncertain operations (network calls, timers, DB calls, listeners) are handled **off the main thread** using browser capabilities and event loop mechanisms.[1]
- JavaScript remains **single-threaded**, but asynchronous handling plus the event loop gives **concurrency-like behavior** without blocking the main thread.[1]

***

## Core Components of the Event Loop Model

The lecture focuses on three key components:[1]

1. **Call Stack**
2. **Browser (Web APIs)**
3. **Callback Queue** (also called **Task Queue**)

### 1. Call Stack

- Structure where JavaScript tracks **function calls**.
- When a function is called, an entry (stack frame) is **pushed** onto the stack.[1]
- When the function finishes, its entry is **popped** from the stack.[1]
- All **synchronous code** is handled directly via the call stack.

### 2. Browser (Web APIs)

- The browser environment provides features like:
  - `setTimeout`
  - DOM events (click, keypress)
  - Network requests (XHR, fetch)
- When asynchronous functions are used, the **work and timers** are handled by the browser, **not** by the JavaScript engine itself.[1]

### 3. Callback Queue (Task Queue)

- A queue where **ready-to-run callbacks** are placed after their timer or event completes.[1]
- Callbacks are added to the **front/back** of this queue as they become eligible (e.g., after timeout completes or event triggers).[1]
- They **do not** run immediately; they must be moved from the callback queue to the call stack by the **event loop**.[1]

***

## How the Event Loop Works

**Event Loop Role:**  
The event loop constantly checks:

1. Is the **call stack empty**?  
2. If yes, is there any callback waiting in the **callback queue**?  
3. If both conditions match, it **moves** the front callback from the queue to the call stack for execution.[1]

In other words:

- Synchronous code is run first on the call stack.[1]
- Asynchronous callbacks wait in the callback queue until:
  - Their condition is ready (timer done or event fired), and
  - The call stack is empty.[1]
- Event loop then transfers callbacks to the call stack one by one.

***

## Example 1: Simple setTimeout Flow

Consider this conceptual code:[1]

```js
console.log("Hi");

setTimeout(sayMyName, 5000);

console.log("End");
```

**Step-by-step behavior:**

1. Execution reaches `console.log("Hi")`:
   - Entry added to call stack.
   - `"Hi"` is printed immediately.
   - Entry removed from call stack.[1]

2. Execution reaches `setTimeout(sayMyName, 5000)`:
   - A frame for `setTimeout` is pushed onto the call stack.[1]
   - Because `setTimeout` is asynchronous and supported by the browser:
     - The timer and callback (`sayMyName`) are **handed over** to the browser.
     - The 5000 ms timer starts in browser APIs.
     - The `setTimeout` frame is popped off the call stack.[1]

3. Execution reaches `console.log("End")`:
   - Entry added to call stack.
   - `"End"` is printed immediately.
   - Entry removed; call stack now empty.[1]

4. After at least 5000 ms:
   - Browser timer completes.
   - The callback `sayMyName` is placed into the **callback queue**.[1]

5. Event loop:
   - Sees the call stack is empty and callback queue has `sayMyName`.
   - Moves `sayMyName` from callback queue to call stack.[1]

6. `sayMyName` executes:
   - Inside, it calls `console.log("Love Babbar")`.
   - `"Love Babbar"` prints.
   - Function completes and is popped from the stack.[1]

**Important points:**

- The callback does **not** run exactly at 5000 ms; it runs after 5000 ms **and** when the call stack is empty.[1]
- This mechanism prevents blocking and allows other synchronous code to run first.

***

## Example 2: Multiple Synchronous + Asynchronous Operations

Conceptual pattern from the lecture (names changed for clarity):[1]

```js
console.log("Love");     // synchronous
console.log("Babbar");   // synchronous

setTimeout(sayMyName, 3000);  // asynchronous

console.log("Manish");   // synchronous
```

**Execution sequence:**

1. `console.log("Love")`:
   - Synchronous; directly prints `"Love"`, stack then cleared.[1]

2. `console.log("Babbar")`:
   - Synchronous; directly prints `"Babbar"`, stack then cleared.[1]

3. `setTimeout(sayMyName, 3000)`:
   - Handed over to browser; 3000 ms timer starts.
   - No blocking of main thread.
   - Stack frame removed.[1]

4. `console.log("Manish")`:
   - Synchronous; directly prints `"Manish"` while the 3-second timer is still running in the browser.
   - Stack cleared.[1]

5. After timer completes:
   - `sayMyName` is added to **callback queue**.[1]

6. Event loop:
   - Call stack is empty.
   - Moves `sayMyName` from callback queue to call stack.
   - Inside `sayMyName`, `"Love Babbar"` is logged synchronously.[1]

**Observed order in console:**

- `Love`  
- `Babbar`  
- `Manish`  
- `Love Babbar` (from the async callback)[1]

This shows that:

- All synchronous code runs **first**, in order.
- Asynchronous code waits until:
  - Its condition is ready (timer done), and
  - Stack is free.[1]

***

## Example 3: Event Listeners and Timers Together

The lecture also demonstrates a case with:

- A **button click handler** (asynchronous event).[1]
- A `setTimeout` with 5000 ms delay.[1]
- Some synchronous console logs.[1]

Conceptual idea:

- Lines 5–7: async code related to a button click listener (on clicking, some function should run).[1]
- Line 9: synchronous `console.log("Hi")`, runs immediately.[1]
- Line 11: `setTimeout` to run a function after 5000 ms (asynchronously).[1]
- Line 15: synchronous log like `"Welcome to loop"`, runs immediately.[1]

**Behavior:**

1. The **event listener setup** is itself handled such that when a click happens, the callback goes to the **callback queue**, not directly to the stack.[1]
2. When the button is clicked:
   - The click event’s callback is enqueued in the callback queue.[1]
   - Event loop then moves it to the call stack when it becomes empty, and it executes (e.g., printing `"You clicked the button"`).[1]
3. Meanwhile, the timer set by `setTimeout` also follows the same pattern:  
   - Timer completes → callback goes to callback queue → event loop moves it to call stack when stack is empty.[1]

This demonstrates that **multiple asynchronous sources** (timers, events) all funnel their callbacks into the **callback queue**, and the event loop coordinates their execution, always respecting that synchronous stack work completes first.[1]

***

## How Event Loop Prevents Blocking

- Asynchronous operations (timers, events, network calls) are **offloaded** to the browser, so the JavaScript engine does not block while waiting.[1]
- The event loop ensures:
  - **All synchronous code** in the call stack completes first.
  - Then callbacks from the callback queue are executed one by one when the stack is empty.[1]
- This avoids blocking on operations whose completion time is **uncertain** (network latency, user interaction), while keeping JavaScript **single-threaded**.[1]

***

## Key Terms (Quick Reference)

- **Synchronous code:** Code that executes immediately when reached; no waiting, runs in order on the call stack.[1]
- **Asynchronous code:** Code whose effect is delayed; uses browser APIs and callbacks (e.g., `setTimeout`, event listeners, network calls).[1]
- **Blocking:** Situation where code prevents further execution until it finishes, causing everything after it to wait.[1]
- **Call Stack:** Data structure where JavaScript tracks active function calls and executes synchronous code.[1]
- **Browser/Web APIs:** Environment provided by the browser to handle timers, events, and network operations.[1]
- **Callback Queue (Task Queue):** Queue where ready callbacks wait for execution after timers/events complete.[1]
- **Event Loop:** Mechanism that repeatedly checks if the call stack is empty and, if so, moves callbacks from the callback queue to the stack for execution.[1]

***

## Summary of Main Takeaways

- JavaScript is **single-threaded**, but still handles **asynchronous behavior** using the event loop model.[1]
- **Synchronous code** executes immediately on the call stack, while **asynchronous operations** are delegated to browser APIs and later return callbacks via the **callback queue**.[1]
- The **event loop** coordinates this by moving callbacks from the queue to the stack **only when the stack is empty**, ensuring all synchronous code finishes first.[1]
- This design prevents **blocking**, keeps the UI responsive, and allows JavaScript to manage concurrency-like behavior in a single thread.